# Meta Features

In [81]:
import numpy as np
import pandas as pd
import datetime
import gc
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import optimize
import seaborn as sns

******

In [84]:
df_merchant = pd.read_csv('merchants.csv')

In [85]:
useful_col = ['merchant_id', 'merchant_group_id', 'merchant_category_id',
              'subsector_id', 'numerical_1', 'numerical_2', 'most_recent_sales_range',
              'most_recent_purchases_range','avg_sales_lag3', 'avg_purchases_lag3',
              'active_months_lag3','avg_sales_lag6', 'avg_purchases_lag6', 'active_months_lag6',
              'avg_sales_lag12', 'avg_purchases_lag12', 'active_months_lag12','category_4']

In [86]:
df_merchant = df_merchant[useful_col]

In [87]:
#df_hist_trans = pd.merge(df_hist_trans,df_merchant,on = 'merchant_id',how='left')

In [88]:
df_new_trans = pd.merge(df_new_trans,df_merchant,on = 'merchant_id',how='left')

In [89]:
df_train = pd.read_csv('train.csv',usecols=['card_id','target'])
df_test = pd.read_csv('test.csv',usecols=['card_id'])

In [90]:
all_set = pd.merge(df_new_trans,df_train,on='card_id',how='left')

In [91]:
all_set.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'category_3', 'merchant_category_id_x', 'merchant_id', 'month_lag',
       'purchase_amount', 'purchase_date', 'category_2', 'state_id',
       'subsector_id_x', 'merchant_group_id', 'merchant_category_id_y',
       'subsector_id_y', 'numerical_1', 'numerical_2',
       'most_recent_sales_range', 'most_recent_purchases_range',
       'avg_sales_lag3', 'avg_purchases_lag3', 'active_months_lag3',
       'avg_sales_lag6', 'avg_purchases_lag6', 'active_months_lag6',
       'avg_sales_lag12', 'avg_purchases_lag12', 'active_months_lag12',
       'category_4', 'target'],
      dtype='object')

In [92]:
for col in ['most_recent_sales_range','most_recent_purchases_range',
            'category_2','category_4','category_3',]:
    arr,_ = pd.factorize(all_set[col])
    all_set[col] = arr

In [93]:
df_train = all_set[~all_set.target.isna()]
df_test = all_set[all_set.target.isna()]

In [94]:
target = df_train['target']

In [95]:
target_copy = target.copy()

In [120]:
target[target_copy<-30] = 1
target[target_copy>-30] = 0

/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/ubuntu/.local/lib/python3.5/site-packages/pandas/core/generic.py:7626: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/ubuntu/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2961: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, 

In [121]:
del df_train['target']

In [122]:
df_train.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'category_3', 'merchant_category_id_x', 'merchant_id', 'month_lag',
       'purchase_amount', 'purchase_date', 'category_2', 'state_id',
       'subsector_id_x', 'merchant_group_id', 'merchant_category_id_y',
       'subsector_id_y', 'numerical_1', 'numerical_2',
       'most_recent_sales_range', 'most_recent_purchases_range',
       'avg_sales_lag3', 'avg_purchases_lag3', 'active_months_lag3',
       'avg_sales_lag6', 'avg_purchases_lag6', 'active_months_lag6',
       'avg_sales_lag12', 'avg_purchases_lag12', 'active_months_lag12',
       'category_4'],
      dtype='object')

In [123]:
useless_features =\
['card_id',
 'merchant_id'
 ]

In [124]:
features = [c for c in df_train.columns if c not in useless_features]
categorical_feats = [c for c in features if 'feature_' in c]

In [126]:
features =\
[
 'installments',
 'category_3',
 'merchant_category_id_x',
 'month_lag',
 'purchase_amount',
 'category_2',
 'subsector_id_x',
 'merchant_group_id',
 'merchant_category_id_y',
 'subsector_id_y',
 'numerical_1',
 'numerical_2',
 'most_recent_sales_range',
 'most_recent_purchases_range',
 'avg_sales_lag3',
 'avg_purchases_lag3',
 'active_months_lag3',
 'avg_sales_lag6',
 'avg_purchases_lag6',
 'active_months_lag6',
 'avg_sales_lag12',
 'avg_purchases_lag12',
 'active_months_lag12',
 'category_4']

In [127]:
len(features)

24

In [128]:
param = {'objective':'binary',
         'num_leaves': 43,
         'min_data_in_leaf': 20,
         'max_depth': -1,
         'learning_rate': 0.15,
         'lambda_l1':0.15,
         "boosting": "gbdt",
         "feature_fraction":0.85,
         'bagging_freq':4,
         "bagging_fraction": 0.9 ,
         "metric": 'binary_logloss',
         "verbosity": -1,
         "random_state": 4}

In [129]:
%%time
folds = KFold(n_splits=2, shuffle=True, random_state=233)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train)):
    print("fold {}".format(fold_))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][features], label=target.iloc[trn_idx])#, categorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx][features], label=target.iloc[val_idx])#, categorical_feature=categorical_feats)

    num_round = 20
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval= 100, early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(df_test[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(mean_squared_error(oof, target)**0.5))

fold 0
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[20]	training's binary_logloss: 0.0346772	valid_1's binary_logloss: 0.0378441
fold 1
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[20]	training's binary_logloss: 0.034146	valid_1's binary_logloss: 0.0380508
CV score: 0.08020 
CPU times: user 27.4 s, sys: 864 ms, total: 28.3 s
Wall time: 6.31 s


In [130]:
train_prediction = pd.DataFrame({"card_id":df_train["card_id"].values})
train_prediction["meta_hist"] = oof

In [131]:
test_prediction = pd.DataFrame({"card_id":df_test["card_id"].values})
test_prediction["meta_hist"] = predictions

In [132]:
meta_train = train_prediction.groupby("card_id").agg({'meta_hist': ['min','sum']})
meta_test = test_prediction.groupby("card_id").agg({'meta_hist': ['min','sum']})

In [133]:
meta_train.columns = ['meta_min', 'meta_sum']
meta_test.columns = ['meta_min', 'meta_sum']

In [134]:
meta_train.to_csv('meta_train.csv')
meta_test.to_csv('meta_test.csv')